In [6]:
import os
# Get the current working directory
cwd = os.getcwd()
print(cwd)

/Users/mariotuci/Desktop/Google-Drive/Master/SoSe-24/Project Studies/Project/Code/latent-communication/utils/federicos stash


In [1]:
from pathlib import Path
import os
import sys
import numpy as np

sys.path.append("../")

import torch
import random
import hydra
from tqdm import tqdm
from utils.dataloaders.dataloader_mnist_single import DataLoaderMNIST
from torchvision.datasets import MNIST
from torchvision.transforms import transforms

from models.definitions.PocketAutoencoder import PocketAutoencoder

from utils.model import load_model, get_transformations
from utils.sampler import simple_sampler, simple_sampler_v1
from utils.visualization import visualize_results

from optimization.optimizer import LinearFitting
from optimization.optimizer import AffineFitting

from optimization.fit_mapping import get_latents

DEVICE = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
augmentations = [transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))]

class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)

In [11]:
DEVICE = torch.device('mps')
model1 = model1 = PocketAutoencoder()
model1.load_state_dict(torch.load('../../models/checkpoints/SMALLAE/MNIST/MNIST_SMLLAE_0.01_32_1_3.pth'))
model1.to(DEVICE)
latentvector, data = simple_sampler_v1(100,model1,augmentations,DEVICE)


In [3]:
print(latentvector.shape)

(100, 49)


In [8]:
order_check = []

pth_files = [file for file in os.listdir('models/checkpoints/SMALLAE/MNIST/') if os.path.isfile(os.path.join('models/checkpoints/SMALLAE/MNIST/', file)) and file.endswith('.pth')]

for file in tqdm(pth_files):
    model_order = []
    latent_space_dict = {}
    model1 = PocketAutoencoder()
    model1.load_state_dict(torch.load(f'models/checkpoints/SMALLAE/MNIST/{file}'))
    model1.to(DEVICE)

    model1.eval()
    DataLoaders = DataLoaderMNIST
    indices = range(60000)  # Assuming you want to use the first 40,000 images
    data_loader = DataLoaderMNIST(128, augmentations, indices=indices)
    test_loader = data_loader.get_test_loader()

    # Get all images from test_loader and convert them to latent space
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(DEVICE)
        with torch.no_grad():
            latent_space = model1.get_latent_space(images)

        # Convert tensor to numpy array
        latent_space_np = latent_space.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()
        
        # Store in dictionary
        for idx in range(batch_idx*128, len(images)):
            latent_space_dict[batch_idx * 128 + idx] = (latent_space_np[idx], labels_np[idx])
            model_order.append(labels_np[idx])
    order_check.append(model_order)
    torch.save(latent_space_dict, 'models/checkpoints/SMALLAE/MNIST/LATENTS/' + str(file).replace('.pth', '_latent_space.pth'))

## Check they are in the same order
for i in range(len(order_check)-1):
    if order_check[i] != order_check[i+1]:
        print('Order is not the same')
        break

print(len(order_check))

100%|██████████| 52/52 [05:28<00:00,  6.33s/it]

52


In [3]:
def create_mapping(cfg, latents1, latents2):
    if cfg.mapping == 'Linear':
        from optimization.optimizer import LinearFitting
        mapping = LinearFitting(latents1, latents2, lamda=cfg.lamda)
    elif cfg.mapping == 'Affine':
        from optimization.optimizer import AffineFitting
        mapping = AffineFitting(latents1, latents2, lamda=cfg.lamda)
    elif cfg.mapping == 'NeuralNetwork':
        from optimization.optimizer import NeuralNetworkFitting
        mapping = NeuralNetworkFitting(latents1, latents2, hidden_dim=cfg.hidden_size, lamda=cfg.lamda, learning_rate=cfg.learning_rate, epochs=cfg.epochs)
    else:
        raise ValueError("Invalid experiment name")
    return mapping

In [4]:

for file1 in os.listdir('../../models/checkpoints/SMALLAE/MNIST/'):
    print(file1)
    if file1.endswith('.pth'):
        
        #latents1 = torch.load('../../models/checkpoints/SMALLAE/MNIST/LATENTS/' + file1)
        for file2 in os.listdir('../../models/checkpoints/SMALLAE/MNIST/'):
            if file1 != file2 and file2.endswith('.pth'):
                
                model1 = PocketAutoencoder()
                model2 = PocketAutoencoder()
                model1.load_state_dict(torch.load('../../models/checkpoints/SMALLAE/MNIST/' + file1))
                model2.load_state_dict(torch.load('../../models/checkpoints/SMALLAE/MNIST/' + file2))
                #latents2 = torch.load('models/checkpoints/SMALLAE/MNIST/LATENTS/' + file2)

                latent1, data = simple_sampler_v1(100,model1,augmentations,DEVICE)
                latent2, data = simple_sampler_v1(100,model2,augmentations,DEVICE)
                
                parameters = {
                    "num_samples": 10,
                    "mapping": "Linear",
                    "lamda": 0.01,
                }
                #latent1_pl = torch.stack([torch.tensor(value[0]) for value in latents1.values()])
                #latent2_pl = torch.stack([torch.tensor(value[0]) for value in latents2.values()])
                cfg = Config(**parameters)
              
                mapping = create_mapping(cfg, latent1, latent2)
                mapping.fit()
                storage_path = f'results/transformations/SMALLAE/{cfg.mapping}_{file1[:-len("_latent_space.pth")]}_{file2[:-len("_latent_space.pth")]}_{cfg.num_samples}'
                Path(storage_path).parent.mkdir(parents=True, exist_ok=True)
                mapping.save_results(storage_path)
                latents1_trafo = mapping.transform(latent1)

                # for n in range(10):
                #     desired_class = n  # Specify the class you want to filter
                #     filtered_samples = []

                #     # Filter samples from the test loader based on the desired class
                #     for data, label in test_loader:
                #         indices = torch.nonzero(label == desired_class, as_tuple=False)
                #         if indices.numel() > 0:
                #             for idx in indices:
                #                 filtered_samples.append((data[idx], label[idx]))

MNIST_SMLLAE_0.001_128_5_4.pth
Defining the problem
Solving the problem
Results saved at  results/transformations/SMALLAE/Linear_MNIST_SMLLAE__MNIST_SMLLAE_10
Defining the problem
Solving the problem
Results saved at  results/transformations/SMALLAE/Linear_MNIST_SMLLAE__MNIST_SMLLAE_0_10
Defining the problem
Solving the problem
Results saved at  results/transformations/SMALLAE/Linear_MNIST_SMLLAE__MNIST_SMLLAE_10
Defining the problem
Solving the problem
Results saved at  results/transformations/SMALLAE/Linear_MNIST_SMLLAE__MNIST_SMLLAE_0_10
Defining the problem
Solving the problem
Failure:interrupted


SolverError: Solver 'SCS' failed. Try another solver, or solve with verbose=True for more information.